In [1]:
{-# LANGUAGE GADTs #-}

In [2]:
import Control.Monad.Trans.Reader

data Expr a where
    Var :: Int -> Expr a
    App :: Expr a -> Expr a -> Expr a
    Lam :: Expr a -> Expr a
    Lit :: a -> Expr a

newtype Eval a = Eval { runEval :: Reader [a] a }

In [3]:
import Debug.Trace

eval :: Expr a -> Eval a
eval expr = case expr of
    Var i -> Eval $ do
        vars <- ask
        pure $ vars !! i
    App f x -> Eval $ do
        x' <- runEval $ eval x
        local (x':) (runEval (eval f))
    Lam f -> eval f
    Lit a -> Eval $ pure a

k = (Lam (Lam (Var 1)))
i = (Lam (Var 0))
s = (Lam (Lam (Lam (App (App (Var 2) (Var 0)) (App (Var 1) (Var 0))))))

i' = App (App s k) k

runReader (runEval (eval k)) [7,5]

5